In [10]:
import os
from typing import Mapping, Optional, Sequence

from dotenv import load_dotenv
import pyspark
from pyspark.sql import SparkSession

# Source
INCOMING_ROOT = "s3a://landing-isis/opralog/incoming"
OPRALOGDB_TABLES: Mapping[str, Optional[Mapping[str, str]]] = dict(
    LOGBOOKS=dict(unique_keys=("LOGBOOK_ID",), partition_by=None),
    LOGBOOK_ENTRIES=dict(
        unique_keys=("LOGBOOK_ID", "ENTRY_ID"), partition_by=None
    ),
    ENTRIES=dict(unique_keys=["ENTRY_ID"], partition_by=None),
    MORE_ENTRY_COLUMNS=dict(
        unique_keys=["ENTRY_ID", "COLUMN_NO", "ENTRY_TYPE_ID"],
        partition_by=None,
    ),
    ADDITIONAL_COLUMNS=dict(
        unique_keys=["COLUMN_NO", "ENTRY_TYPE_ID"], partition_by=None
    ),
)

# Destination
TARGET_CATALOG = "isis"
TARGET_DB = "cleaned"
OPRALOG_LOGBOOK_ENTRY = "opralog_logbook_entry"
OPRALOG_LOGBOOK_ENTRY_COMMENT = "opralog_logbook_entry_comment"
load_dotenv()

True

In [11]:
spark = (
    SparkSession.builder
        .master("spark://data-accelerator.isis.cclrc.ac.uk:7077")
        .config("spark.hadoop.fs.s3a.access.key", os.environ["S3_ACCESS_KEY"])
        .config("spark.hadoop.fs.s3a.secret.key", os.environ["S3_ACCESS_SECRET"])
        .getOrCreate()
)
spark.active()

In [12]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {TARGET_CATALOG}.{TARGET_DB}")
spark.sql(f"USE {TARGET_CATALOG}.{TARGET_DB}")

""


In [19]:
%%time

# Load in temporary tables
loadtype = "full"
ingest_date = "2024/11/21"
for tablename in OPRALOGDB_TABLES.keys():
    sources = f"{INCOMING_ROOT}/{tablename}/{loadtype}/{ingest_date}/*.parquet"
    print(f"Ingesting path '{sources}'")
    df = spark.read.parquet(sources)
    df.createOrReplaceTempView(tablename)
    print(spark.sql(f"SELECT COUNT(*) FROM {tablename}"))

Ingesting path 's3a://landing-isis/opralog/incoming/LOGBOOKS/full/2024/11/21/*.parquet'
+--------+
|count(1)|
+--------+
|      47|
+--------+

Ingesting path 's3a://landing-isis/opralog/incoming/LOGBOOK_ENTRIES/full/2024/11/21/*.parquet'
+--------+
|count(1)|
+--------+
|   48998|
+--------+

Ingesting path 's3a://landing-isis/opralog/incoming/ENTRIES/full/2024/11/21/*.parquet'
+--------+
|count(1)|
+--------+
|   41663|
+--------+

Ingesting path 's3a://landing-isis/opralog/incoming/MORE_ENTRY_COLUMNS/full/2024/11/21/*.parquet'
+--------+
|count(1)|
+--------+
|  511095|
+--------+

Ingesting path 's3a://landing-isis/opralog/incoming/ADDITIONAL_COLUMNS/full/2024/11/21/*.parquet'
+--------+
|count(1)|
+--------+
|    1716|
+--------+

CPU times: user 8.57 ms, sys: 7.73 ms, total: 16.3 ms
Wall time: 3.81 s


In [20]:
%%time

table_ensure_exists = f"""
CREATE TABLE IF NOT EXISTS {OPRALOG_LOGBOOK_ENTRY} (
  entry_id LONG,
  logbook_id LONG,
  extra_column_no LONG,
  extra_column_id LONG,
  logbook_name STRING,
  time_logged TIMESTAMP,
  description STRING,
  column_title STRING,
  string_data STRING,
  number_data DOUBLE
)
USING iceberg
PARTITIONED BY (logbook_name, month(time_logged))
"""
spark.sql(table_ensure_exists)

CPU times: user 92 μs, sys: 2.1 ms, total: 2.19 ms
Wall time: 19.9 ms


""


In [21]:
%%time

snapshot_name = "snapshot_without_comments"
snapshot_ddl = f"""
CREATE OR REPLACE TEMPORARY VIEW {snapshot_name} AS (
    SELECT
      CAST(ENTRIES.ENTRY_ID AS LONG) AS entry_id,
      CAST(LOGBOOKS.LOGBOOK_ID AS LONG) AS logbook_id,
      CAST(ADDITIONAL_COLUMNS.COLUMN_NO AS LONG) AS extra_column_no,
      CAST(ADDITIONAL_COLUMNS.ENTRY_TYPE_ID AS LONG) AS extra_column_id,
      CAST(TRIM(LOGBOOK_NAME) AS STRING) AS logbook_name,
      TIME_LOGGED AS time_logged,
      CAST(TRIM(ENTRY_DESCRIPTION) AS STRING) AS description,
      CAST(TRIM(COL_TITLE) AS STRING) AS column_title,
      CAST(TRIM(COL_DATA) AS STRING) AS string_data,
      CAST(NUMBER_VALUE AS DOUBLE) AS number_data 
    FROM ENTRIES
    JOIN LOGBOOK_ENTRIES ON LOGBOOK_ENTRIES.ENTRY_ID = ENTRIES.ENTRY_ID
    JOIN LOGBOOKS ON LOGBOOKS.LOGBOOK_ID = LOGBOOK_ENTRIES.LOGBOOK_ID
    LEFT OUTER JOIN MORE_ENTRY_COLUMNS ON MORE_ENTRY_COLUMNS.ENTRY_ID = ENTRIES.ENTRY_ID
    LEFT OUTER JOIN ADDITIONAL_COLUMNS ON ADDITIONAL_COLUMNS.COLUMN_NO = MORE_ENTRY_COLUMNS.COLUMN_NO AND ADDITIONAL_COLUMNS.ENTRY_TYPE_ID = MORE_ENTRY_COLUMNS.ENTRY_TYPE_ID
    WHERE
      LOGBOOK_ENTRIES.LOGBOOK_ID = PRINCIPAL_LOGBOOK
      AND (COL_DATA IS NOT NULL OR NUMBER_VALUE IS NOT NULL)
)
"""
spark.sql(snapshot_ddl)

CPU times: user 402 μs, sys: 2.36 ms, total: 2.76 ms
Wall time: 42.4 ms


""


In [22]:
%%time

merge_ddl = f"""
MERGE INTO {OPRALOG_LOGBOOK_ENTRY} t
USING {snapshot_name} s
ON t.logbook_id = s.logbook_id AND t.entry_id = s.entry_id AND t.extra_column_no = s.extra_column_no AND t.extra_column_id = s.extra_column_id
WHEN NOT MATCHED THEN INSERT *
"""
spark.sql(merge_ddl)


CPU times: user 7.48 ms, sys: 1.6 ms, total: 9.08 ms
Wall time: 21.3 s


""


In [23]:
spark.sql(f"DROP VIEW {snapshot_name}")
del snapshot_ddl
del snapshot_name

In [24]:
%%time

snapshot_name = "snapshot_comments"
snapshot_ddl = f"""
CREATE OR REPLACE TEMPORARY VIEW {snapshot_name} AS (
    SELECT
      CAST(ENTRIES.ENTRY_ID AS LONG) AS entry_id,
      CAST(TRIM(SHADOW_COMMENT) AS STRING) AS comment_text
    FROM ENTRIES
)
"""
spark.sql(snapshot_ddl)

CPU times: user 0 ns, sys: 811 μs, total: 811 μs
Wall time: 12.2 ms


""


In [25]:
%%time

table_ensure_exists = f"""
CREATE TABLE IF NOT EXISTS {OPRALOG_LOGBOOK_ENTRY_COMMENT} (
  entry_id LONG,
  comment_text STRING
)
USING iceberg
"""
spark.sql(table_ensure_exists)

CPU times: user 1.69 ms, sys: 98 μs, total: 1.79 ms
Wall time: 12.8 ms


""


In [26]:
%%time

merge_snapshot = f"""
MERGE INTO {OPRALOG_LOGBOOK_ENTRY_COMMENT} t
USING {snapshot_name} s
ON t.entry_id = s.entry_id
WHEN NOT MATCHED THEN INSERT *
"""
spark.sql(merge_snapshot)

CPU times: user 1.38 ms, sys: 1.18 ms, total: 2.56 ms
Wall time: 2.37 s


""


In [27]:
spark.sql(f"DROP VIEW {snapshot_name}")
del merge_snapshot
del snapshot_name